# 前処理機能のデモ

このノートでは、`src/utils/preprocessing.py` で実装した各機能を用いて合成データから特徴量を計算し、図で確認します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import preprocessing
from src.utils.config_utils import get_preprocessing_params

params = get_preprocessing_params()
window_size = params['window_size']
fs = params['sampling_rate']

t = np.linspace(0, window_size/fs, window_size)
acc = np.stack([
    np.sin(2*np.pi*1*t),
    np.sin(2*np.pi*0.5*t),
    np.cos(2*np.pi*1.5*t)
], axis=1)
quat = np.tile([1,0,0,0], (window_size,1))
df = pd.DataFrame(acc, columns=['acc_x','acc_y','acc_z'])
for i,col in enumerate(['rot_w','rot_x','rot_y','rot_z']):
    df[col] = quat[:,i]


In [ ]:
# world linear acceleration
df[['lin_acc_x','lin_acc_y','lin_acc_z']] = [
    preprocessing.linear_acceleration(row[['acc_x','acc_y','acc_z']].values,
                                      row[['rot_w','rot_x','rot_y','rot_z']].values)
    for _, row in df.iterrows()
]

# compute statistics and peaks
window = df[['lin_acc_x','lin_acc_y','lin_acc_z']].values[np.newaxis,:,:]
stats = preprocessing.compute_basic_statistics(window)
peaks = preprocessing.compute_peak_features(window)
fft_energy = preprocessing.compute_fft_band_energy(window, fs=fs, bands=params['fft_bands'])


In [ ]:
# plot linear acceleration
plt.figure(figsize=(6,3))
plt.plot(t, df['lin_acc_x'], label='lin_acc_x')
plt.xlabel('Time [s]')
plt.ylabel('Acceleration')
plt.title('Linear Acceleration Example')
plt.legend()
plt.show()

# plot statistics as bar chart
plt.figure(figsize=(6,3))
labels = [f'stat_{i}' for i in range(stats.shape[1])]
plt.bar(labels, stats[0])
plt.title('Basic Statistics')
plt.xticks(rotation=45)
plt.show()

# plot FFT energy
plt.figure(figsize=(6,3))
for i, band in enumerate(params['fft_bands']):
    plt.bar(i, fft_energy[0, i], label=f'{band[0]}-{band[1]}Hz')
plt.title('FFT Band Energy')
plt.ylabel('Power')
plt.xticks(range(len(params['fft_bands'])), [f'{b[0]}-{b[1]}Hz' for b in params['fft_bands']])
plt.show()
